In [ ]:
### temas a consultar
"""

Modelo KMV (Kecholfer, McQuown y Vasicek)
Probit y Logit
survival analysis
PCA Curves
Score de credito
CDS, ABS, MBS, CDO

"""

In [ ]:
import math

def kmv_model(equity_value, total_debt, risk_free_rate, volatility, time_to_maturity):
    """
    Calculates the probability of default using the KMV model.
    :param equity_value: The market value of equity
    :param total_debt: The face value of total debt outstanding
    :param risk_free_rate: The annual risk-free interest rate
    :param volatility: The annualized volatility of the equity
    :param time_to_maturity: Time to maturity of the debt in years
    :return: The probability of default
    """
    d1 = (math.log(equity_value / total_debt) + (risk_free_rate + (0.5 * volatility ** 2)) * time_to_maturity) / (volatility * math.sqrt(time_to_maturity))
    d2 = d1 - (volatility * math.sqrt(time_to_maturity))
    pd = 1 - norm.cdf(d1) - ((total_debt / equity_value) * norm.cdf(d2))
    return pd

This function calculates the probability of default using the KMV model, which is a structural credit risk model that relates the market value of a firm's assets to its total debt outstanding. The inputs to the model are the market value of equity (equity_value), the face value of total debt outstanding (total_debt), the annual risk-free interest rate (risk_free_rate), the annualized volatility of the equity (volatility), and the time to maturity of the debt in years (time_to_maturity). The function returns the probability of default (pd).

The KMV model uses the Black-Scholes-Merton option pricing model to estimate the probability of default. The function calculates the d1 and d2 parameters using the standard Black-Scholes-Merton formulas, and then uses these to calculate the probability of default using the cumulative distribution function of the standard normal distribution (norm.cdf()).

Note that you will need to import the math module and the norm object from the scipy.stats module for this function to work.

## 1. Modelación de Curva Yield con PCA para Valoración de Riesgo de Mercado


Esta sección veremos cómo los componentes principales calculados en una serie temporal multivariada de curvas de rendimiento para EE.UU. permiten identificar el tipo los tipos de movimientos más comunes  que ocurren en las tasas de interés a diferentes vencimientos. El paper seminal de esta sección es **Litterman, R., & Scheinkman, J. (1991). Common factors affecting bond returns. Journal of fixed income**

### 1.1 Teoría: Curva Yield

In [ ]:
import pandas as pd
import numpy as np
import fredapi
import plotly.express as px

#pip install fredapi
#pip install fred

# load yield curve data for the 10-year Treasury constant maturity rate from FRED
fred = fredapi.Fred(api_key='')
yield_curve_3m = fred.get_series('DTB3', start_date='2010-01-01').dropna()
yield_curve_1y = fred.get_series('DGS1', start_date='2010-01-01').dropna()
yield_curve_2y = fred.get_series('DGS2', start_date='2010-01-01').dropna()
yield_curve_5y = fred.get_series('DGS5', start_date='2010-01-01').dropna()
yield_curve_10y = fred.get_series('DGS10', start_date='2010-01-01').dropna()
yield_curve_20y = fred.get_series('DGS20', start_date='2010-01-01').dropna()
yield_curve = pd.DataFrame({'3m': yield_curve_3m,
                            '1y': yield_curve_1y,
                            '2y': yield_curve_2y,
                            '5y': yield_curve_5y,
                            '10y': yield_curve_10y,
                            '20y': yield_curve_20y}).loc["2000-01-01":].dropna()

In [10]:
import pandas as pd
yield_curve=pd.read_csv("fred_data.csv")
yield_curve["Date"]=pd.to_datetime(yield_curve["Date"])
yield_curve.set_index("Date",inplace=True)

In [12]:
# Veamos la curva en diferentes cortes de tiempo

yield_curve_2010=yield_curve.loc["2010-12-15"]
yield_curve_2020=yield_curve.loc["2020-03-30"]
yield_curve_2023=yield_curve.loc["2023-01-30"]
yield_cross=pd.concat([yield_curve_2010,yield_curve_2020,yield_curve_2023],axis=1)
yield_cross.columns=["2010","2020","2023"]
fig = px.line(yield_cross,title='Yield Curves in Time',markers=True)
fig.show()

In [ ]:
# Veamos el comportamiento Historico de cada nodo
fig = px.line(yield_curve,title='Yield Nodes in Time')
fig.show()

Para el propósito del análisis actual, no será relevante tanto la forma particular que asuma la curva de rendimiento en un determinado día, sino que nos enfocaremos los cambios absolutos entre dos puntos consecutivos en el tiempo. Estos movimientos en la curva resulta útiles no  solo para el análisis del riesgo de interés, sino también para determinar el allocation en un portafolio que podrían aprovechar dichos movimientos.

In [ ]:
#Veamos la correlación de la estructura a plazos de la tasa de interés
yield_curve.corr()

Observamos que la estructura  está altamente correlacionada, en particular, entre tasas de vencimiento cercano. Las cifras en la tabla muestran   que la correlación entre los rendimientos es mayor alrededor de la diagonal principal, lo que significa que las tasas de interés de maturity similar se mueven de manera cercana. Por el contrario, la correlación es más débil entre las tasas de interés de vencimientos mas lejanos.

In [ ]:
#Veamos la correlación de la estructura a plazos de la tasa de interés
fig=px.imshow(yield_curve.corr(), text_auto=True, aspect="auto",color_continuous_scale='RdBu_r')
fig.show()

In [ ]:
#Diferencias dia a dis en puntos basicos
df_basis=yield_curve.diff().dropna()*100

#Estandarizamos y centramos los datos
df_standard=(df_basis-df_basis.mean())/(df_basis.std())
df_standard


In [ ]:
#Create a covariance matrix 
cov_matrix_array = np.array(np.cov(df_standard, rowvar=False))

# Perform eigendecomposition
eigenvalues, eigenvectors = np.linalg.eig(cov_matrix_array)

# Put data into a DataFrame 
df_eigval = pd.DataFrame({"Eigenvalues":eigenvalues})
df_eigvec = pd.DataFrame(eigenvectors)
df_eigvec.columns=["v1","v2","v3","v4","v5","v6"]
df_eigvec.index=["3m","1y","2y","5y","10y","20y"]
# Veamos el comportamiento Historico de cada nodo
fig = px.line(df_eigvec.iloc[:,0:3],markers=True)
fig.show()

In [ ]:
# Work out explained proportion 
df_eigval["Explained proportion"] = df_eigval["Eigenvalues"] / np.sum(df_eigval["Eigenvalues"])

#Format as percentage
df_eigval

In [ ]:
principal_components = df_basis.dot(eigenvectors)
principal_components.columns=["PC1","PC2","PC3","PC4","PC5","PC6"]
fig = px.line(principal_components.iloc[:,0:3])
fig.show()

In [ ]:
principal_components["slope"]=df_basis["10y"]-df_basis["2y"]


In [ ]:
fig = px.line(principal_components[["PC2","slope"]].iloc[0:100,:])
fig.show()


In [ ]:
# Perform eigendecomposition
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
factors = pca.fit_transform(df_standard)


In [ ]:
factors

In [ ]:
pca.explained_variance_ratio_
pca.singular_values_

In [ ]:
changes

In [ ]:
yield_curve.dropna().isnull().sum()

In [ ]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

# example yield curve data for multiple nodes (10 nodes, 8 maturities each)
yield_curves = pd.DataFrame(np.random.randn(8, 10), columns=list('ABCDEFGHIJ'))

In [ ]:
# calculate changes in yield rates for each maturity and node
yields = yield_curves.values
changes = np.diff(yields, axis=0) / yields[:-1, :]

# use PCA to extract level and slope factors
pca = PCA(n_components=2)
factors = pca.fit_transform(changes.T)

# plot time series of slope and level factors
fig, ax = plt.subplots(nrows=2, ncols=1, figsize=(10, 8))
ax[0].plot(factors[:, 0])
ax[0].set_title('Level Factor')
ax[1].plot(factors[:, 1])
ax[1].set_title('Slope Factor')
plt.show()

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
# Leemos la data de la ruta o repositorio
df = pd.read_csv("spot_curve_uk.csv")
# Select all of the data up to 10 years

df.set_index("Date",inplace=True)
df.head()

### Probit

Cuyo nombre proviene de la abreviación de **probability unit**

Pasaremos a modelar ahora situaciones donde buscamos un indicador o conocer si un resultado se ha dado. Modelaremos entonces probabilidades usando herramientas econometricas para hacer proposiciones probabilisticas acerca de la ocurrencia de estos eventos. 

Así estos modelos a estudiar son inherentemente e intrinsecamente no lineales. Empezaremos así con resultados binarios 

### Models for binary outcomes

Construiremos modelos que relacionan un resultado a un conjunto de factores. Nuestra aproximación buscará analizar cada uno de ellos en el marco general de un modelo de probabilidad:

Prob(event j occurs | **x**) = Prob(Y = j | **x**) = F(relevant effects, parameters, **x**)

Prob(Y = 1| **x**) = Probability that event of interest occurs| **x**,

y, naturalmente, Prob(Y = 0| x) = [1 - Prob(Y = 1| x)] es la probabilidad de que el evento no ocurra. 

$Prob(Y = 1| x) =\int_{-\infty }^{x'B}\phi(t)\mathrm{d} t=\Phi(x'B)$

The question of which distribution to use is a natural one. The logistic distribution
is similar to the normal except in the tails, which are considerably heavier. (It more
closely resembles a t distribution with seven degrees of freedom.) For intermediate
values of xB, the two distributions tend to give very similar probabilities. The
logistic distribution tends to give larger probabilities to Y = 1 when xB is extremely
small (and smaller probabilities to Y = 1 when xB is very large) than the normal
distribution. It is difficult to provide practical generalities on this basis, however,
as they would require knowledge of B. We might expect different predictions from
the two models, however, if the sample contains (1) very few responses (Y’s equal
to 1) or very few nonresponses (Y’s equal to 0) and (2) very wide variation in an
important independent variable, particularly if (1) is also true. There are practical
reasons for favoring one or the other in some cases for mathematical convenience, but
it is difficult to justify the choice of one distribution or another on theoretical grounds.
Amemiya (1981) discusses a number of related issues, but as a general proposition,
the question is unresolved. In most applications, the choice between these two seems
not to make much difference.

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.tools as sm
from statsmodels.discrete.discrete_model import Probit
data = pd.read_csv('https://vincentarelbundock.github.io/Rdatasets/csv/carData/Mroz.csv')

In [ ]:
data = data.drop(data.columns[0], axis = 1)
data["lfp"] = data["lfp"] == "yes"
data["wc"] = data["wc"] == "yes"
data["hc"] = data["hc"] == "yes"

In [ ]:
Y = data["lfp"]
X = data.drop(["lfp"], 1)
X = sm.add_constant(X)
model = Probit(Y, X.astype(float))
probit_model = model.fit()
print(probit_model.summary())

In [ ]:
mfx = probit_model.get_margeff(at="all")
#print(mfx.summary())
mfx

In [ ]:
import statsmodels.api as sm
import matplotlib.pyplot as plt
import numpy as np

# Generate some example data
np.random.seed(0)
nobs = 100
#x1 = c
x2 = np.random.normal(size=nobs)

# Add small amount of random noise to the predictors to avoid perfect separation
x1 = np.random.normal(size=nobs)+ np.random.normal(scale=0.1, size=nobs)
x2 =  np.random.normal(scale=0.1, size=nobs)

X = sm.add_constant(np.column_stack((x1, x2)))
y = (x1 + x2 > 0)

# Fit the logit model
model = sm.Logit(y, X).fit()

# Define the range of x values for which to calculate the predictions
x_vals = np.linspace(x1.min(), x1.max(), 100)

# Calculate the marginal effect of x1 at the mean of x2
mean_x2 = x2.mean()
x_grid = np.column_stack((x_vals, np.repeat(mean_x2, 100)))
predict_mean_x2 = model.predict(exog=x_grid)

# Plot the marginal effect of x1 at the mean of x2
plt.plot(x_vals, predict_mean_x2)
plt.xlabel('x1')
plt.ylabel('P(y=1)')
plt.title('Marginal Effect of x1 at the Mean of x2')
plt.show()